<a href="https://colab.research.google.com/github/shirleyzz/cap5610_machineleanrning/blob/master/Assignment2/HW2(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HW2(2)-using data augmentation

## Loading the CIFAR10 data set

In [0]:
import keras 
import numpy as np
from random import seed
from random import randrange
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers
from keras.callbacks import LearningRateScheduler
from keras.datasets import cifar10
import matplotlib.pyplot as plt

In [0]:
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

## Preprocessing

In [0]:
# train_images = train_images.reshape((50000, 32 * 32 * 3))
train_images = train_images.astype('float32') / 255
# test_images = test_images.reshape((10000, 32 * 32 * 3))
test_images = test_images.astype('float32') / 255
num_classes = 10
train_labels = keras.utils.to_categorical(train_labels,num_classes)
test_labels = keras.utils.to_categorical(test_labels,num_classes)

In [0]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')
validation_datagen = ImageDataGenerator(rescale=1./255)

In [14]:
#network architecture 
network = Sequential()
network.add(Conv2D(32, (3, 3), padding='same',input_shape=train_images[0].shape))
network.add(Activation('softmax'))
network.add(Flatten())
network.add(Dense(512))
network.add(Activation('relu'))
network.add(Dropout(0.5))
network.add(Dense(10))
network.add(Activation('softmax'))
network.summary()
#compilation step
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)
network.compile(optimizer=opt,
                loss='categorical_crossentropy',
                metrics=['accuracy'])
#train the model
epochs = 30

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_4 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 32768)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               16777728  
_________________________________________________________________
activation_5 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                5130      
__________

In [0]:
#split into validation set and training set at the rate of 8:2
i = 40000
training_images = train_images[:i]
valid_images = train_images[:-i]
training_labels = train_labels[:i]
valid_labels = train_labels[:-i]
#
train_datagen.fit(training_images)
validation_datagen.fit(valid_images)

In [17]:
history = network.fit_generator(train_datagen.flow(training_images, training_labels,
                                     batch_size=128),
                        steps_per_epoch=100,
                        epochs=epochs,
                        validation_data=(valid_images, valid_labels),
                        validation_steps=50)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/30
100/100 [==============================] - 11s 108ms/step - loss: 2.3097 - acc: 0.0969 - val_loss: 2.3026 - val_acc: 0.0944
Epoch 2/30
100/100 [==============================] - 9s 89ms/step - loss: 2.3026 - acc: 0.0961 - val_loss: 2.3026 - val_acc: 0.1028
Epoch 3/30
100/100 [==============================] - 9s 89ms/step - loss: 2.3026 - acc: 0.0952 - val_loss: 2.3026 - val_acc: 0.1064
Epoch 4/30
100/100 [==============================] - 9s 90ms/step - loss: 2.3026 - acc: 0.0972 - val_loss: 2.3026 - val_acc: 0.1095
Epoch 5/30
100/100 [==============================] - 9s 90ms/step - loss: 2.3026 - acc: 0.1008 - val_loss: 2.3026 - val_acc: 0.1043
Epoch 6/30
100/100 [==============================] - 9s 91ms/step - loss: 2.3026 - acc: 0.0981 - val_loss: 2.3026 - val_acc: 0.0951
Epoch 7/30
100/100 [==============================] - 9s 90ms/step - loss:

In [18]:
scores = network.evaluate(test_images, test_labels, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 2s 154us/step
Test loss: 2.3027018138885498
Test accuracy: 0.1059
